In [1]:

from gensim.models import LdaModel
from gensim.corpora import Dictionary, MmCorpus
import numpy as np
import pandas as pd
    
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

#import bokeh.plotting as bp
#from bokeh.plotting import save
#from bokeh.models import HoverTool

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#!pip install pyLDAvis --user
try:
    import pyLDAvis.sklearn
except:
    !pip install --user pyLDAvis
finally:
    import pyLDAvis.sklearn
    
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

%config InlineBackend.figure_format = 'retina'

import warnings
warnings.filterwarnings("ignore")

## First LDA attempt. Note: intresting clusterization of badwods!

In [4]:
df = pd.read_pickle('../data/df_preprocessed_eng_remap.pckle')
df.head()

,description,event_id,category,category_label,lang,lang_reliab,desc_stemm,desc_lemm,desc_stemm_no_badwords,desc_lemm_no_badwords,remap_category,remap_category_label
0,"<p><span>Hey explorersssss, what's up?</span><...",239719250,language/ethnic identity,15,English,99,secret spot hong kong night view food adventur...,secret spot hong kong night view food adventur...,secret spot hong kong night view food adventur...,secret spot hong kong night view food adventur...,language/ethnic identity,11
1,"<p>Free, unauditioned, collaborative pop choir...",gpjktmywhbnb,music,18,English,99,sing us free unaudit collabor pop choir identi...,sing u free unauditioned collaborative pop cho...,sing us unaudit collabor pop choir identifi wo...,sing u unauditioned collaborative pop choir id...,music,13
2,"<p>We provide a forum to learn about, promote ...",drrtzmywhbgb,career/business,3,English,98,east valley busi network meetup provid forum l...,east valley business network meetup provide fo...,east valley busi network provid forum learn pr...,east valley business network provide forum lea...,career/business,1
3,<p><b>【WhyNot!?JAPAN + MeetUp Collaboration Ev...,239719229,socializing,27,English,92,friendli friday whynot japan meetup collabor e...,friendly friday whynot japan meetup collaborat...,friendli whynot japan collabor whynot japan si...,friendly whynot japan collaboration whynot jap...,socializing,20
4,<p>This is an introductory meeting to get to k...,mtzxwmywjbjb,tech,30,English,99,authent option introductori meet get know fell...,authentication option introductory meet get kn...,authent option introductori get fellow coder s...,authentication option introductory get fellow ...,tech,22


### Vectorization

In [6]:
tf_vectorizer = TfidfVectorizer(analyzer = "word", max_features = 1000 )

vectorized = tf_vectorizer.fit_transform(df.desc_lemm)
vectorized.shape

vectorized.A

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### LDA model with 32 topics

In [7]:
df.remap_category.unique().shape

(24,)

In [8]:

lda_model = LatentDirichletAllocation(n_components=24, random_state=0, n_jobs=40)

X_topics = lda_model.fit_transform(vectorized)

word_dict = {}
for topic_idx, topic in enumerate(lda_model.components_):
    word_dict[topic_idx] = [tf_vectorizer.get_feature_names()[i] for i in topic.argsort()[:-10 - 1:-1]]

pd.DataFrame(word_dict).T

,0,1,2,3,4,5,6,7,8,9
0,movie,night,pm,happy,beer,hour,drink,tuesday,meet,wednesday
1,workout,fitness,class,training,body,get,exercise,water,fun,session
2,life,learn,workshop,help,work,change,relationship,health,self,talk
3,data,project,talk,use,developer,learn,technology,design,work,cloud
4,business,estate,real,marketing,investor,market,learn,property,strategy,invest
5,walk,coffee,park,tea,meet,garden,bring,picnic,let,lunch
6,ticket,music,wine,event,band,dance,pm,night,live,party
7,dance,salsa,english,class,lesson,language,spanish,conversation,pm,beginner
8,business,network,event,group,professional,networking,woman,meeting,meet,member
9,sing,song,study,church,child,music,god,sunday,service,group


## LDA VIZ

In [9]:
pyLDAvis.sklearn.prepare(lda_model, vectorized, tf_vectorizer, mds='pcoa') #see cluster 17

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
11     0.055178  0.127902       1        1  7.523937
21    -0.095065 -0.000806       2        1  7.255531
23    -0.058746 -0.006815       3        1  6.540061
2      0.074331  0.062785       4        1  6.065887
19    -0.084606 -0.008699       5        1  5.433049
22     0.007760 -0.029268       6        1  5.403185
3      0.129385 -0.009036       7        1  5.153209
13    -0.005180  0.159959       8        1  4.997590
6     -0.059091 -0.058570       9        1  4.978234
14     0.007546 -0.030285      10        1  4.869986
4      0.122831 -0.026925      11        1  4.494814
16    -0.061742 -0.062782      12        1  3.808099
8      0.091220 -0.047925      13        1  3.748066
18    -0.011308  0.044323      14        1  3.601232
7     -0.025542  0.035844      15        1  3.303022
1     -0.037209  0.099134      16        1  3.122677
15     0.020984  0.014976      17        1  3.111595
12    -0.090636 -0.055359      18        1  3.105845
5     -0.087537 -0.016358      19        1  3.078546
0     -0.095164 -0.080787      20        1  2.713471
9      0.004507  0.041269      21        1  2.241127
17    -0.080176 -0.024061      22        1  2.139355
10     0.075150 -0.035161      23        1  1.802245
20     0.203110 -0.093354      24        1  1.509238, topic_info=    Category         Freq         Term        Total  loglift  logprob
997  Default  1911.000000         yoga  1911.000000  30.0000  30.0000
147  Default  3706.000000        class  3706.000000  29.0000  29.0000
109  Default  2026.000000     business  2026.000000  28.0000  28.0000
211  Default  2142.000000        dance  2142.000000  27.0000  27.0000
357  Default  2360.000000         game  2360.000000  26.0000  26.0000
..       ...          ...          ...          ...      ...      ...
539  Topic24    75.087843       meetup  2714.737104   0.6058  -4.8466
429  Topic24    58.057500  information  1084.056228   1.2665  -5.1038
935  Topic24    60.087081          use  1833.053879   0.7756  -5.0695
899  Topic24    62.505995         time  2970.674824   0.3323  -5.0300
109  Topic24    59.459654     business  2026.099699   0.6650  -5.0800

[1177 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
4         1  0.015767  account
4         2  0.055185  account
4         3  0.063069  account
4         4  0.039418  account
4         5  0.137963  account
...     ...       ...      ...
999      20  0.014662    young
999      21  0.108502    young
999      22  0.049852    young
999      23  0.014662    young
999      24  0.002932    young

[14148 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 22, 24, 3, 20, 23, 4, 14, 7, 15, 5, 17, 9, 19, 8, 2, 16, 13, 6, 1, 10, 18, 11, 21])

In [17]:
p = pyLDAvis.sklearn.prepare(lda_model, vectorized, tf_vectorizer, mds='mmds') #30 32 31 19


In [ ]:
pyLDAvis.save_html(p, 'lda.html')

In [11]:
pyLDAvis.sklearn.prepare(lda_model, vectorized, tf_vectorizer, mds='mmds') #30 32 31 19

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
11    -0.145027  0.047143       1        1  7.523937
21     0.143127 -0.129297       2        1  7.255531
23    -0.013198 -0.099963       3        1  6.540061
2     -0.066397  0.075371       4        1  6.065887
19     0.093933 -0.106384       5        1  5.433049
22     0.070472  0.046657       6        1  5.403185
3     -0.062151  0.137711       7        1  5.153209
13    -0.170515 -0.040908       8        1  4.997590
6      0.042424 -0.069043       9        1  4.978234
14     0.029070  0.019730      10        1  4.869986
4     -0.025210  0.147097      11        1  4.494814
16     0.072844 -0.037953      12        1  3.808099
8      0.035035  0.137150      13        1  3.748066
18    -0.077429 -0.031064      14        1  3.601232
7     -0.087317 -0.122300      15        1  3.303022
1     -0.133831 -0.089063      16        1  3.122677
15    -0.047901  0.008140      17        1  3.111595
12     0.060735 -0.129977      18        1  3.105845
5      0.126674 -0.038077      19        1  3.078546
0      0.166981 -0.005918      20        1  2.713471
9      0.014673  0.061275      21        1  2.241127
17    -0.001551 -0.161227      22        1  2.139355
10     0.142557  0.179414      23        1  1.802245
20    -0.167999  0.201485      24        1  1.509238, topic_info=    Category         Freq         Term        Total  loglift  logprob
997  Default  1911.000000         yoga  1911.000000  30.0000  30.0000
147  Default  3706.000000        class  3706.000000  29.0000  29.0000
109  Default  2026.000000     business  2026.000000  28.0000  28.0000
211  Default  2142.000000        dance  2142.000000  27.0000  27.0000
357  Default  2360.000000         game  2360.000000  26.0000  26.0000
..       ...          ...          ...          ...      ...      ...
539  Topic24    75.087843       meetup  2714.737104   0.6058  -4.8466
429  Topic24    58.057500  information  1084.056228   1.2665  -5.1038
935  Topic24    60.087081          use  1833.053879   0.7756  -5.0695
899  Topic24    62.505995         time  2970.674824   0.3323  -5.0300
109  Topic24    59.459654     business  2026.099699   0.6650  -5.0800

[1177 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
4         1  0.015767  account
4         2  0.055185  account
4         3  0.063069  account
4         4  0.039418  account
4         5  0.137963  account
...     ...       ...      ...
999      20  0.014662    young
999      21  0.108502    young
999      22  0.049852    young
999      23  0.014662    young
999      24  0.002932    young

[14148 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 22, 24, 3, 20, 23, 4, 14, 7, 15, 5, 17, 9, 19, 8, 2, 16, 13, 6, 1, 10, 18, 11, 21])

In [12]:
pyLDAvis.sklearn.prepare(lda_model, vectorized, tf_vectorizer, mds='tsne') #32

PreparedData(topic_coordinates=               x          y  topics  cluster      Freq
topic                                                 
11     -2.094312  -4.917393       1        1  7.523937
21     -7.050441 -26.933765       2        1  7.255531
23      0.907884 -26.890028       3        1  6.540061
2       9.496101 -27.258984       4        1  6.065887
19    -10.379341 -20.326855       5        1  5.433049
22     12.774957 -20.149075       6        1  5.403185
3      -2.654498 -20.358215       7        1  5.153209
13    -16.854658 -14.290071       8        1  4.997590
6      -6.784276 -13.591986       9        1  4.978234
14    -11.555417  -6.092376      10        1  4.869986
4       7.125036  -4.617568      11        1  4.494814
16     17.465633 -12.304852      12        1  3.808099
8       0.619269 -41.903828      13        1  3.748066
18     10.562428 -39.387993      14        1  3.601232
7      -9.209205 -40.105782      15        1  3.303022
1       4.577722 -33.698727      16        1  3.122677
15      4.958429 -20.040026      17        1  3.111595
12     20.007538 -23.483959      18        1  3.105845
5     -13.726171 -31.711887      19        1  3.078546
0       0.975135 -13.024300      20        1  2.713471
9     -18.955805 -24.389320      21        1  2.241127
17     -3.471285 -33.626064      22        1  2.139355
10      8.854472 -12.951131      23        1  1.802245
20     16.073193 -32.058128      24        1  1.509238, topic_info=    Category         Freq         Term        Total  loglift  logprob
997  Default  1911.000000         yoga  1911.000000  30.0000  30.0000
147  Default  3706.000000        class  3706.000000  29.0000  29.0000
109  Default  2026.000000     business  2026.000000  28.0000  28.0000
211  Default  2142.000000        dance  2142.000000  27.0000  27.0000
357  Default  2360.000000         game  2360.000000  26.0000  26.0000
..       ...          ...          ...          ...      ...      ...
539  Topic24    75.087843       meetup  2714.737104   0.6058  -4.8466
429  Topic24    58.057500  information  1084.056228   1.2665  -5.1038
935  Topic24    60.087081          use  1833.053879   0.7756  -5.0695
899  Topic24    62.505995         time  2970.674824   0.3323  -5.0300
109  Topic24    59.459654     business  2026.099699   0.6650  -5.0800

[1177 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
4         1  0.015767  account
4         2  0.055185  account
4         3  0.063069  account
4         4  0.039418  account
4         5  0.137963  account
...     ...       ...      ...
999      20  0.014662    young
999      21  0.108502    young
999      22  0.049852    young
999      23  0.014662    young
999      24  0.002932    young

[14148 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 22, 24, 3, 20, 23, 4, 14, 7, 15, 5, 17, 9, 19, 8, 2, 16, 13, 6, 1, 10, 18, 11, 21])

# Check after "badwords" cleansing

In [13]:
tf_vectorizer = TfidfVectorizer(analyzer = "word", stop_words='english', max_features = 1000 )

vectorized = tf_vectorizer.fit_transform(df.desc_lemm_no_badwords)
vectorized.shape

vectorized.A

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [14]:
lda_model = LatentDirichletAllocation(n_components=24, random_state=0, n_jobs=40)

X_topics = lda_model.fit_transform(vectorized)

word_dict = {}
for topic_idx, topic in enumerate(lda_model.components_):
    word_dict[topic_idx] = [tf_vectorizer.get_feature_names()[i] for i in topic.argsort()[:-10 - 1:-1]]

pd.DataFrame(word_dict).T

,0,1,2,3,4,5,6,7,8,9
0,meeting,topic,discussion,talk,monthly,discuss,location,open,idea,share
1,party,drink,single,prepay,dress,night,bar,mingle,purchase,code
2,yoga,class,body,mat,flow,practice,movement,level,strength,exercise
3,meditation,practice,mind,yoga,class,experience,mindfulness,stress,life,body
4,dinner,food,comedy,restaurant,menu,drink,vegan,cook,dish,eat
5,class,course,art,learn,workshop,student,basic,skill,training,session
6,business,network,estate,real,startup,learn,investor,professional,marketing,entrepreneur
7,coffee,english,language,conversation,write,spanish,french,chat,tea,practice
8,ride,hike,trail,mile,bike,park,run,route,pace,road
9,game,play,player,team,night,soccer,board,league,tournament,fun


In [15]:
pyLDAvis.sklearn.prepare(lda_model, vectorized, tf_vectorizer, mds='pcoa') #see cluster 17

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
6     -0.074400  0.103047       1        1  8.186521
8      0.111253 -0.062423       2        1  7.017964
13    -0.151129 -0.036906       3        1  6.419197
21    -0.107029  0.105424       4        1  5.444174
14     0.089268  0.032550       5        1  5.085780
18    -0.082711  0.040316       6        1  4.923744
10     0.104259 -0.038235       7        1  4.845354
9      0.092267  0.013127       8        1  4.736499
3     -0.165024 -0.073486       9        1  4.673863
22     0.109745  0.003412      10        1  4.355034
4      0.090324  0.043172      11        1  4.168888
2     -0.101374 -0.152255      12        1  4.021753
19     0.005491  0.057336      13        1  4.019155
0     -0.010391  0.112309      14        1  3.790775
1      0.126639  0.045606      15        1  3.533756
5     -0.102225 -0.018472      16        1  3.396901
16    -0.026333 -0.016153      17        1  3.382191
7     -0.015974  0.080500      18        1  3.263406
20    -0.048923 -0.072562      19        1  3.201015
23    -0.001189 -0.063203      20        1  3.033810
15     0.078277 -0.063483      21        1  2.870540
11     0.112395 -0.037506      22        1  2.547116
17    -0.046861  0.143403      23        1  1.781160
12     0.013645 -0.145519      24        1  1.301403, topic_info=    Category         Freq        Term        Total  loglift  logprob
218  Default  2299.000000       dance  2299.000000  30.0000  30.0000
997  Default  2105.000000        yoga  2105.000000  29.0000  29.0000
150  Default  4089.000000       class  4089.000000  28.0000  28.0000
359  Default  2617.000000        game  2617.000000  27.0000  27.0000
534  Default  1969.000000  meditation  1969.000000  26.0000  26.0000
..       ...          ...         ...          ...      ...      ...
794  Topic24    35.318507     session  2033.334089   0.2887  -5.2893
545  Topic24    29.666526        mind  1151.513391   0.6829  -5.4637
968  Topic24    27.745096     weather   727.090823   1.0757  -5.5307
283  Topic24    29.345279       enjoy  1378.486233   0.4921  -5.4746
515  Topic24    30.610930        make  2653.569821  -0.1206  -5.4324

[1078 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
6         1  0.057671    act
6         2  0.026913    act
6         3  0.046137    act
6         4  0.019224    act
6         5  0.107652    act
...     ...       ...    ...
999      18  0.016683  young
999      19  0.114003  young
999      20  0.008342  young
999      22  0.019464  young
999      23  0.016683  young

[13782 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 9, 14, 22, 15, 19, 11, 10, 4, 23, 5, 3, 20, 1, 2, 6, 17, 8, 21, 24, 16, 12, 18, 13])

In [16]:
pyLDAvis.sklearn.prepare(lda_model, vectorized, tf_vectorizer, mds='mmds') #30 32 31 19

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
6      0.110515  0.128950       1        1  8.186521
8     -0.174327 -0.060853       2        1  7.017964
13    -0.030661  0.198432       3        1  6.419197
21     0.116896  0.187599       4        1  5.444174
14     0.016609 -0.142127       5        1  5.085780
18     0.042986  0.100368       6        1  4.923744
10    -0.080710 -0.064136       7        1  4.845354
9      0.113138 -0.115971       8        1  4.736499
3     -0.081294  0.201295       9        1  4.673863
22    -0.027342 -0.090458      10        1  4.355034
4      0.046776 -0.119047      11        1  4.168888
2     -0.166442  0.147827      12        1  4.021753
19     0.032191  0.003085      13        1  4.019155
0      0.120230  0.038347      14        1  3.790775
1      0.039569 -0.223891      15        1  3.533756
5     -0.024685  0.123374      16        1  3.396901
16    -0.033806  0.078514      17        1  3.382191
7      0.145093 -0.010633      18        1  3.263406
20    -0.128235  0.071927      19        1  3.201015
23    -0.134968 -0.169949      20        1  3.033810
15     0.242210  0.009677      21        1  2.870540
11    -0.079278 -0.133358      22        1  2.547116
17     0.258074 -0.099119      23        1  1.781160
12    -0.322539 -0.059852      24        1  1.301403, topic_info=    Category         Freq        Term        Total  loglift  logprob
218  Default  2299.000000       dance  2299.000000  30.0000  30.0000
997  Default  2105.000000        yoga  2105.000000  29.0000  29.0000
150  Default  4089.000000       class  4089.000000  28.0000  28.0000
359  Default  2617.000000        game  2617.000000  27.0000  27.0000
534  Default  1969.000000  meditation  1969.000000  26.0000  26.0000
..       ...          ...         ...          ...      ...      ...
794  Topic24    35.318507     session  2033.334089   0.2887  -5.2893
545  Topic24    29.666526        mind  1151.513391   0.6829  -5.4637
968  Topic24    27.745096     weather   727.090823   1.0757  -5.5307
283  Topic24    29.345279       enjoy  1378.486233   0.4921  -5.4746
515  Topic24    30.610930        make  2653.569821  -0.1206  -5.4324

[1078 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
6         1  0.057671    act
6         2  0.026913    act
6         3  0.046137    act
6         4  0.019224    act
6         5  0.107652    act
...     ...       ...    ...
999      18  0.016683  young
999      19  0.114003  young
999      20  0.008342  young
999      22  0.019464  young
999      23  0.016683  young

[13782 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 9, 14, 22, 15, 19, 11, 10, 4, 23, 5, 3, 20, 1, 2, 6, 17, 8, 21, 24, 16, 12, 18, 13])